<a href="https://colab.research.google.com/github/riemann1859/store-item-demand-forecasting-challenge/blob/master/Lasso%2BDirectStrategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#import necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#load datasets
df_train=pd.read_csv('train.csv',sep=',')
df_test=pd.read_csv('test.csv',sep=',')

#drop id column 

if 'id' in df_test.columns:
  df_test.drop(labels='id',axis=1,inplace=True)
  
if 'id' in df_train.columns:
  df_train.drop(labels='id',axis=1,inplace=True)
  

# create sales column for df_test which is to be predicted  
df_test['sales']=np.NaN

df_test['purpose']='test'
df_train['purpose']='train'

#combine df_train and df_test into one dataframe

df=pd.concat([df_train, df_test], axis=0)


#convert to date_time object

df['date'] = pd.to_datetime(df['date'])

df=df.sort_values(by=['item','store','date'])

In [0]:
# prepare dataframe 
# if we denote sales column with S_t, we add S_(t-1), S_(t-2),.... to this new dataframe

new_df=pd.DataFrame()
for item in df.item.unique():
    for store in df.store.unique():
        #add lagged sales column to  the related part of the original dataframe 
        new=pd.concat([df.loc[(df.store==store)&(df.item==item)]]+[df.loc[(df.store==store)&(df.item==item),'sales'].shift(i) for i in range(1,470)],axis=1)
        new.columns=list(df.columns)+['sales_lagged_{}'.format(i) for i in range(1,470)]
        new_df=pd.concat([new_df,new],axis=0)

In [0]:
#create some new categorical features from date

new_df['weekofyear']=new_df.date.apply(lambda x:x.weekofyear)
new_df['dayofweek']=new_df.date.apply(lambda x:x.dayofweek)
new_df['year']=new_df.date.apply(lambda x:x.year)
new_df['month']=new_df.date.apply(lambda x:x.month)

new_df['weekofyear']=new_df['weekofyear'].astype('category')
new_df['dayofweek']=new_df['dayofweek'].astype('category')
new_df['year']=new_df['year'].astype('category')
new_df['month']=new_df['month'].astype('category')
new_df['store']=new_df.store.astype('category')
new_df['item']=new_df.item.astype('category')

In [0]:
# convert categoricals to dummy variables

new_df_with_dummies=pd.concat([pd.get_dummies(new_df.drop(labels='purpose',axis=1)),new_df.purpose],axis=1)




In [11]:
from sklearn.linear_model import  LassoLarsIC


preds=pd.DataFrame(data=np.full((45000,), np.NaN),columns=['sales'])

# here we set up 90 models
# to forecast i-step ahead we set up a new  Lasso model for each i
# this means that for every model we construct a new training data

for i in range(1,91):
    
    columns_to_be_dropped=['sales_lagged_{}'.format(k) for k in range(1,i)]
    df_i=new_df_with_dummies.drop(labels=columns_to_be_dropped, axis=1)
    
    train_i=df_i[df_i.purpose=='train']
    test_i=df_i[df_i.purpose=='test']

    train_i.dropna(inplace=True)


    xtrain_i=train_i.drop(labels=['sales','date','purpose'],axis=1)
    ytrain_i=train_i.sales.values
    
    

    model_aic = LassoLarsIC(criterion='aic')
    model_aic.fit(xtrain_i,ytrain_i)
    
    preds.loc[(test_i.index%90)==(i-1),'sales']=model_aic.predict(test_i.iloc[(test_i.index%90)==(i-1),2:-1].values)

    
    


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [16]:
from sklearn.linear_model import  LassoLarsIC


preds_bic=pd.DataFrame(data=np.full((45000,), np.NaN),columns=['sales'])


for i in range(1,91):
    
    columns_to_be_dropped=['sales_lagged_{}'.format(k) for k in range(1,i)]
    df_i=new_df_with_dummies.drop(labels=columns_to_be_dropped, axis=1)
    
    train_i=df_i[df_i.purpose=='train']
    test_i=df_i[df_i.purpose=='test']

    train_i.dropna(inplace=True)


    xtrain_i=train_i.drop(labels=['sales','date','purpose'],axis=1)
    ytrain_i=train_i.sales.values
    
    

    model_bic = LassoLarsIC(criterion='bic')
    model_bic.fit(xtrain_i,ytrain_i)
    
    preds_bic.loc[(test_i.index%90)==(i-1),'sales']=model_bic.predict(test_i.iloc[(test_i.index%90)==(i-1),2:-1].values)

    
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/i

In [0]:
from google.colab import files

with open('preds_bic.csv', 'w') as f:
  preds_bic.to_csv(f)

files.download('preds_bic.csv')